In [72]:
import pandas as pd
import numpy as np

**train_mfti.parquet** – сырые данные, которые можно использовать для обучения модели

* event_date – дата взаимодейтсвия
* event_timestamp – timestamp взаимодействия в секундах 
* vacancy_id_ - id вакансии, с которой было взаимодействие
* cookie_id – id пользователя по его браузеру/ip/устройству
* user_id – id пользователя на сайте rabota.ru (есть только для зарегистрированных пользователей)
* event_type – тип взаимодействия

In [6]:
train_mfti_df = pd.read_parquet('Data_Rabota/train_mfti.parquet', engine='pyarrow')
train_mfti_df

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type
0,2022-08-01,1659323026,129850,97990f1a021d4be19aa3f955b7eacab4,951f53de61764ea0b51317200a0dbbfc,show_vacancy
1,2022-08-01,1659377255,108347,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
2,2022-08-01,1659376695,109069,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
3,2022-08-01,1659376722,171425,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
4,2022-08-01,1659374929,252384,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
...,...,...,...,...,...,...
12292583,2022-08-11,1660246479,162851,0d3c58c6864546c689fa0997de484b30,18124998fd644e80b667a1d84bf63846,preview_click_response
12292584,2022-08-11,1660246106,136818,ef559274a81e4ff4b92aa3e9f6805886,5c71c1f1b7cf492ba05871c400a05250,preview_click_response
12292585,2022-08-11,1660250139,143721,c4453128ae584dca8f2a8d20a7c74263,33c968125459420e89ffbc8cffece317,preview_click_response
12292586,2022-08-11,1660250318,154423,c4453128ae584dca8f2a8d20a7c74263,33c968125459420e89ffbc8cffece317,preview_click_response


In [7]:
train_mfti_df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12292588 entries, 0 to 12292587
Data columns (total 6 columns):
 #   Column           Non-Null Count     Dtype 
---  ------           --------------     ----- 
 0   event_date       12292588 non-null  object
 1   event_timestamp  12292588 non-null  int64 
 2   vacancy_id_      12292588 non-null  int64 
 3   cookie_id        12292588 non-null  object
 4   user_id          8711177 non-null   object
 5   event_type       12292588 non-null  object
dtypes: int64(2), object(4)
memory usage: 562.7+ MB


In [8]:
# Выводим информацию о количестве событий каждого типа

train_mfti_df['event_type'].value_counts()

show_vacancy              6198889
preview_click_vacancy     4781280
click_response             384090
click_contacts             277584
preview_click_response     190635
click_favorite             155844
preview_click_favorite     107016
preview_click_contacts     102050
click_phone                 79191
preview_click_phone         16009
Name: event_type, dtype: int64

In [9]:
# Выводим информацию о количестве событийв каждый день

train_mfti_df['event_date'].value_counts()

2022-09-05    292929
2022-09-06    288203
2022-09-07    278221
2022-09-19    278194
2022-09-13    276529
               ...  
2022-08-12     98335
2022-08-14     88921
2022-08-13     58818
2022-08-28     15492
2022-08-27     13727
Name: event_date, Length: 62, dtype: int64

In [51]:
# Выводим информацию о количестве уникальных значений для всех колонок

print(f"nUnique vacancy_id_: {train_mfti_df['vacancy_id_'].nunique()}")
print(f"nUnique event_date: {train_mfti_df['event_date'].nunique()}")
print(f"nUnique cookie_id: {train_mfti_df['cookie_id'].nunique()}")
print(f"nUnique user_id: {train_mfti_df['user_id'].nunique()}")

nUnique vacancy_id_: 160167
nUnique event_date: 62
nUnique cookie_id: 330180
nUnique user_id: 209335


In [25]:
# Проверяем, всегда ли одному cookie_id соответствует один user_id.
# НЕТ

train_mfti_df.groupby(['cookie_id'])['user_id'].nunique().value_counts()

1     215167
0     113830
2       1124
3         47
4          5
5          4
56         1
8          1
7          1
Name: user_id, dtype: int64

In [26]:
# Проверяем, всегда ли одному user_id соответствует один cookie_id.
# НЕТ

train_mfti_df.groupby(['user_id'])['cookie_id'].nunique().value_counts()

1     201813
2       6883
3        529
4         81
5         16
6          6
7          4
10         1
8          1
15         1
Name: cookie_id, dtype: int64

In [62]:
train_mfti_df.groupby(['cookie_id'])['event_timestamp'].count()

cookie_id
0000c4548c3944c08972bbdc1fa4eb85    15
0000d7508334414ca792c5ff66eb8c14    24
0000e63f423542fe8090575b611cea4d    31
00013bf3e58244b48d4dbdf43543c30c    11
00015019b0594984afc3fd9fa9557490    81
                                    ..
ffff3581b7e7405db25fac5c5dd1bff9    15
ffff42fe199b46d59ce38bde9c95b148    80
ffff8283d29644e69ad6d08cc3ca7fb7    14
ffffd6d4f2134dd6aa80ff71e942b508    27
ffffdb17f45b4032b386d691d52e6c00    12
Name: event_timestamp, Length: 330180, dtype: int64

**test_public_mfti.parquet** – часть теста, с открытым таргетом, для проверки работоспособности решений.

* cookie_id - id пользователя по его браузеру/ip/устройству
* vacancy_id_ - список вакансий, на которые пользователь откликнулся или позвонил в течение месяца после окончания данных train

In [14]:
test_public_mfti_df = pd.read_parquet('Data_Rabota/test_public_mfti.parquet', engine='pyarrow')
test_public_mfti_df

,cookie_id,vacancy_id_
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812..."
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348..."
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065..."
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]"
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]"
...,...,...
767,fdbcda17f22f406486837059e76c7fed,"[207851, 254989, 213344, 214180, 222146]"
768,fe6193ab26494ace9be5aae36e507618,"[115352, 230546, 225527, 120188, 109360, 23212..."
769,fe95b2826ee1452b81201ed3f4c3294d,"[240362, 114852, 253946, 251081, 127546, 244688]"
770,ff1aef256a49481698bb2e938510ff36,"[231194, 236363, 220747, 244688, 100094, 24052..."


In [19]:
test_public_mfti_df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 772 entries, 0 to 771
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   cookie_id    772 non-null    object
 1   vacancy_id_  772 non-null    object
dtypes: object(2)
memory usage: 12.2+ KB


In [38]:
# В train датафрейме
# 1. Группируем события по cookie_id
# 2. Собираем номера вакансий в событиях в список
# 3. Оставляем только уникальные номера вакансий

# Нужно для того, чтобы смерджить с test_public

train_cookie_vac = train_mfti_df.groupby(['cookie_id'])['vacancy_id_'].unique().apply(list).rename('vacancy_id_train')
train_cookie_vac

cookie_id
0000c4548c3944c08972bbdc1fa4eb85     [174953, 176171, 187529, 153975, 137659, 182445]
0000d7508334414ca792c5ff66eb8c14    [108690, 192850, 230581, 258036, 250327, 10667...
0000e63f423542fe8090575b611cea4d    [226832, 251362, 122287, 137592, 141592, 15828...
00013bf3e58244b48d4dbdf43543c30c     [209629, 250729, 135961, 138688, 138502, 243795]
00015019b0594984afc3fd9fa9557490    [141563, 132841, 127352, 126251, 115924, 11348...
                                                          ...                        
ffff3581b7e7405db25fac5c5dd1bff9    [102780, 164602, 207423, 210628, 235862, 23989...
ffff42fe199b46d59ce38bde9c95b148    [129564, 132245, 242429, 258148, 258060, 10880...
ffff8283d29644e69ad6d08cc3ca7fb7             [115949, 127277, 182846, 182877, 231900]
ffffd6d4f2134dd6aa80ff71e942b508    [155987, 102931, 171308, 139419, 156033, 15931...
ffffdb17f45b4032b386d691d52e6c00             [150843, 107893, 107125, 185412, 160164]
Name: vacancy_id_train, Length: 330180, dtyp

In [39]:
# Мерджим по cookie_id и переименовываем колонку для ясности

new_df = test_public_mfti_df.merge(train_cookie_vac, left_on='cookie_id', right_on='cookie_id')
new_df = new_df.rename(columns={'vacancy_id_': 'vacancy_id_public'})
new_df

,cookie_id,vacancy_id_public,vacancy_id_train
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812...","[258338, 258095, 134006, 243946, 182454, 10109..."
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348...","[130142, 250327, 249949, 212411, 213439, 25711..."
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065...","[203680, 256529, 228348, 191293, 101949, 21387..."
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]","[174953, 252106, 178415, 180382, 171602, 18472..."
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]","[248787, 107486, 162242, 167168, 114852, 15021..."
...,...,...,...
767,fdbcda17f22f406486837059e76c7fed,"[207851, 254989, 213344, 214180, 222146]","[170333, 103567, 243294, 212971, 229314]"
768,fe6193ab26494ace9be5aae36e507618,"[115352, 230546, 225527, 120188, 109360, 23212...","[104054, 181381, 159326, 124712, 154533, 12195..."
769,fe95b2826ee1452b81201ed3f4c3294d,"[240362, 114852, 253946, 251081, 127546, 244688]","[195754, 119075, 208350, 108093, 103582, 10793..."
770,ff1aef256a49481698bb2e938510ff36,"[231194, 236363, 220747, 244688, 100094, 24052...","[101241, 217788, 217615, 215110, 210229, 19246..."


In [45]:
# Пишем функцию чтобы проверить, нет ли пересечений в списках вакансий из public и train

def check_intersections(column_1, column_2):
    set_1 = set(column_1)
    set_2 = set(column_2)
    if len(set_1.intersection(set_2)) > 0:
        return 1
    else:
        return 0

In [48]:
check_intersect = new_df.apply(lambda x: check_intersections(x.vacancy_id_public, x.vacancy_id_train), axis=1)
check_intersect

0      0
1      0
2      0
3      0
4      0
      ..
767    0
768    0
769    0
770    0
771    0
Length: 772, dtype: int64

In [49]:
# Пересечений нет

sum(check_intersect)

0

In [ ]:
# Убираем из train датасета cookie_id, совпадающие с public

# !!!TODO!!! - Ушли почти все строки, возможно стоит подумать над разделением public на 2 части: test и для итоговой валидации

In [59]:
train_mfti_df_fixed = train_mfti_df[train_mfti_df['cookie_id'].isin(new_df['cookie_id'])]
train_mfti_df_fixed

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type
1710,2022-08-01,1659351159,231108,434ddd68eaea4cceafb83708dec13b18,e4fcc60c6f50486bb7461f09b029fd90,preview_click_vacancy
1711,2022-08-01,1659349035,231108,434ddd68eaea4cceafb83708dec13b18,e4fcc60c6f50486bb7461f09b029fd90,preview_click_vacancy
1967,2022-08-01,1659335799,251198,8581291f23c14bdba86f9a557937cb9f,None,preview_click_vacancy
2040,2022-08-01,1659351345,254843,2fa1d53ca1cb4eafa8c3fb15d96d5af7,88e332e8e3c648dcb534ed927e09bf8a,preview_click_vacancy
2255,2022-08-01,1659363592,109659,560be80587e547f99ef177fb0063d17f,None,preview_click_vacancy
...,...,...,...,...,...,...
12291777,2022-08-11,1660168490,109977,2af1e3a0093d4faeb2142ee8ca840800,133f8215404944009a9ca3c9517df39a,preview_click_response
12291947,2022-08-11,1660230927,101194,5f253d039b59457fa6809124909a3682,086c95d82a614aeea379d304206b2055,preview_click_response
12291967,2022-08-11,1660195337,136266,acd0c1ec6e444622a940077b0e10a970,b6ff3dcf9a1a453983c568f59dea4fc2,preview_click_response
12292154,2022-08-11,1660205708,197331,4049280d091b4428969dae790a5ec10d,1140b1457b8f4b379b6e59211306d70e,preview_click_response


In [70]:
# Создаем колонку с рейтингом действия: 1 - целевые действия, 0 - остальные

purpose_events=['click_response',
                'preview_click_response',
                'click_contacts',
                'preview_click_contacts',
                'click_phone',
                'preview_click_phone']

def check_is_purpose(event):
    if event in purpose_events:
        return 1
    else:
        return 0

train_mfti_df_fixed['rating'] = train_mfti_df_fixed['event_type'].apply(check_is_purpose)
train_mfti_df_fixed

C:\Users\Yury\AppData\Local\Temp\ipykernel_13448\1434335322.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_mfti_df_fixed['rating'] = train_mfti_df_fixed['event_type'].apply(check_is_purpose)


,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type,rating
1710,2022-08-01,1659351159,231108,434ddd68eaea4cceafb83708dec13b18,e4fcc60c6f50486bb7461f09b029fd90,preview_click_vacancy,0
1711,2022-08-01,1659349035,231108,434ddd68eaea4cceafb83708dec13b18,e4fcc60c6f50486bb7461f09b029fd90,preview_click_vacancy,0
1967,2022-08-01,1659335799,251198,8581291f23c14bdba86f9a557937cb9f,None,preview_click_vacancy,0
2040,2022-08-01,1659351345,254843,2fa1d53ca1cb4eafa8c3fb15d96d5af7,88e332e8e3c648dcb534ed927e09bf8a,preview_click_vacancy,0
2255,2022-08-01,1659363592,109659,560be80587e547f99ef177fb0063d17f,None,preview_click_vacancy,0
...,...,...,...,...,...,...,...
12291777,2022-08-11,1660168490,109977,2af1e3a0093d4faeb2142ee8ca840800,133f8215404944009a9ca3c9517df39a,preview_click_response,1
12291947,2022-08-11,1660230927,101194,5f253d039b59457fa6809124909a3682,086c95d82a614aeea379d304206b2055,preview_click_response,1
12291967,2022-08-11,1660195337,136266,acd0c1ec6e444622a940077b0e10a970,b6ff3dcf9a1a453983c568f59dea4fc2,preview_click_response,1
12292154,2022-08-11,1660205708,197331,4049280d091b4428969dae790a5ec10d,1140b1457b8f4b379b6e59211306d70e,preview_click_response,1


In [80]:
# Группируем по пользователям, вакансиям и рейтингам чтобы получить матрицу с рейтингами

# !!!TODO!!! - Пока группирую пользователей по cookie_id, исправить, когда будет новый user_id

# !!!TODO!!! - Матрица вышла очень разреженная, для большинства пользователей и вакансий нет пересечений.
#              Нужно подумать, как это правильно задавать

rating_matrix = pd.pivot_table(train_mfti_df_fixed, values='rating', index=['vacancy_id_'],
                       columns=['cookie_id'], aggfunc=np.sum)
rating_matrix

cookie_id,000cd76cd33f43d4a1ac1d16d10f8bf7,0034bc7f404341ba8412665453e7825a,00a6c5a64a274c55a836402bdeb3b2c4,015937a125b14e74bdff1cddc49f9172,01de50c280794cec8804f16f45f847b7,020c7d1d9a44451c9ef107fc2fe726f5,02411265feae492ebca615b5b03e8bba,028b4b7fd44b438b8b2b6198a74a8c88,029c6b8042064d0899fd73f3290da565,02db6a09b03f4459b77f0fa0acce6b42,...,fce9ee168f264ace8ecf4f65c79479ad,fd5c778538f64fc2bf79f34444e378b4,fd7f92d7d36341048204363adb39a305,fd9b749117874e609c689f3ad2a219ae,fdbaa0e092f54f1d96ac7a83618c77a0,fdbcda17f22f406486837059e76c7fed,fe6193ab26494ace9be5aae36e507618,fe95b2826ee1452b81201ed3f4c3294d,ff1aef256a49481698bb2e938510ff36,ff40591f76e34013a61b8af400002a07
vacancy_id_,,,,,,,,,,,,,,,,,,,,,
100002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260154,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**test_private_users_mfti.parquet** – часть теста, с закрытым таргетом для итоговой проверки решений

* cookie_id - id пользователя по его браузеру/ip/устройству

In [15]:
test_private_users_mfti_df = pd.read_parquet('Data_Rabota/test_private_users_mfti.parquet', engine='pyarrow')
test_private_users_mfti_df

,cookie_id
0,0018914ba3e54011b28fa715583d3354
1,0035c298d8c64f368ae730a9cca9bb20
2,00956458877448ec9fba87fb97443fdf
3,0099387c921b41e7bae6c99dd8254b60
4,009f65e8ae99413a8da94a491320580a
...,...
3081,ffadd195859444d2ade2479b0611c5c1
3082,ffbc08b528c64f22996873fc63872202
3083,ffdeaf3c34544529880aebf17c103f6c
3084,ffefa79a74804ee69e6c131e0d05b948


In [16]:
test_private_users_mfti_df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3086 entries, 0 to 3085
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   cookie_id  3086 non-null   object
dtypes: object(1)
memory usage: 24.2+ KB


**test_private_sample_submission_mfti.parquet** – файл с примером предсказаний, который требуется получить по итогу хакатона 

* cookie_id - id пользователя по его браузеру/ip/устройству
* predictions – список из 5 id вакансий, которые модель предсказала как наиболее релеватные для данного пользователя

In [17]:
test_private_sample_submission_mfti_df = pd.read_parquet('Data_Rabota/test_private_sample_submission_mfti.parquet', engine='pyarrow')
test_private_sample_submission_mfti_df

,cookie_id,predictions
0,0018914ba3e54011b28fa715583d3354,"[100100, 100101, 100102, 100103, 100104]"
1,0035c298d8c64f368ae730a9cca9bb20,"[100100, 100101, 100102, 100103, 100104]"
2,00956458877448ec9fba87fb97443fdf,"[100100, 100101, 100102, 100103, 100104]"
3,0099387c921b41e7bae6c99dd8254b60,"[100100, 100101, 100102, 100103, 100104]"
4,009f65e8ae99413a8da94a491320580a,"[100100, 100101, 100102, 100103, 100104]"
...,...,...
3081,ffadd195859444d2ade2479b0611c5c1,"[100100, 100101, 100102, 100103, 100104]"
3082,ffbc08b528c64f22996873fc63872202,"[100100, 100101, 100102, 100103, 100104]"
3083,ffdeaf3c34544529880aebf17c103f6c,"[100100, 100101, 100102, 100103, 100104]"
3084,ffefa79a74804ee69e6c131e0d05b948,"[100100, 100101, 100102, 100103, 100104]"


In [18]:
test_private_sample_submission_mfti_df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3086 entries, 0 to 3085
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   cookie_id    3086 non-null   object
 1   predictions  3086 non-null   object
dtypes: object(2)
memory usage: 48.3+ KB
